프로그래밍에서 관용구는 특정 작업을 수행하기 위해 코드를 작성하는 특별한 방법이다 매번 동일한 구조를 반복하고 따르는 것이 일반적이다. 디자인 패턴과는 다르다. 가장 큰 차이점은 디자인 패턴을 언어와 무관한 고차원의 개념으로 코드로 즉시 변환되지 않느다. 반면 관용구는 실제 코딩으로 변환된다. 

관용구는 코드이므로 언어에 따라 다르다. 관용구를 따르는 코드를 관용적이라 부르고 특히 파이썬에서는 pythonic이라고 한다.

관용적인 방식으로 코드를 작성했을 때 일반적으로 더 나은 성능을 낸다. 코드도 더 작고 이해하기 쉽다.

전체 개발팀이 동일한 패턴과 구조에 익숙해지면 실수를 줄이고 문제의 본질에 보다 집중할 수 있다.

### 인덱스와 슬라이스

파이썬은 음수 인덱스가 가능하다.

In [2]:
nums = [1, 2, 3, 4, 5]
nums[-1]

5

slice를 이용하여 특정 구간의 요소를 구할 수 있다.

In [3]:
nums[2:5]

[3, 4, 5]

In [4]:
nums[:3]

[1, 2, 3]

In [5]:
nums[3:]

[4, 5]

In [6]:
nums[::]

[1, 2, 3, 4, 5]

In [7]:
nums[1:-1:2]

[2, 4]

위 모든 예제에서 시퀀스에 간격을 전달할 때 실제로는 슬라이스를 전달하는 것과 같다.
슬라이스는 파이썬 내장 객체로 직접 빌드하여 전달할 수도 있다.

In [8]:
interval = slice(1, -1 , 2)
nums[interval]

[2, 4]

In [9]:
interval = slice(None, 3)
nums[interval]

[1, 2, 3]

### 자체 시퀀스 생성

방금 설명한 기능은 "\__getitem\__"이라는 매직 메서드 덕분에 동작한다. 이것은 myobject[key]와 같은 형태를 사용할 때 호출되는 메서드로 key에 해당하는 대괄호 안의 값을 파라미터로 전달한다. 시퀀스는 '\__getitem\__'과 '\__len\__' 모두 구현하는 객체임로 반복이 가능하다. 리스트, 튜플과 문자열은 표준 라이브러리에 있는 시퀀스 객체의 예이다. 

이 섹션에서는 시퀀스나 이터러블 객체를 만들지 않고 키로 객체의 특정 요소를 가져 오는 방법에 대해 다룬다.

클래스가 표준 라이브러리 객체를 감싸는 래퍼인 경우 기본 객체에 가능한 많은 동작을 위임할 수 있다. 

In [11]:
class items:
    def __init__(self, *values):
        self._values = list(values)
    
    def __len__(self):
        return len(self._values)
    
    def __getitem__(self, item):
        return self._values.__getitem__(item)

이 예제는 캡슐화 방식을 사용했다. 다른 방법으로 상속을 사용할 수도 있다.

### 컨텍스트 관리자(context manager)

컨텍스트 관리자가 유용한 이유는 패턴에 잘 대응되기 때문이다. 이 패턴은 사실상 모든 코드에 적용될 수 있으며 사전조건과 사후조건을 가지고 있다. 즉 주요 동작의 전후에 작업을 실행하려고 할 때 유용하다.

with 문은 컨텍스트 관리자로 진입하게 한다. 컨텍스트 관리자는 '\__enter\__'와 '\__exit\__' 두 개의 매직 메서드로 구성된다. with문은 '\__enter\__' 메서드를 호출하고 이 메서드가 무엇을 반환하든 as 이후에 지정된 변수에 할당된다. 사실 '\__enter\__' 메서드가 특정한 값을 반환할 필요는 없다. 설사 값을 반환한다 하더라도 피요하지 않으면 변수에 할당하지 않아도 된다.

해당 블록에 대한 마지막 문장이 끝나면 컨텍스트가 종료되며 이는 파이썬이 처음 호출한 원래 컨텍스트 관리자 객체의 '\__exit\__' 메서드를 호출함을 의미한다. 

In [16]:
def stop_database():
    run("systemctl stop postgresql.service")
    
def start_database():
    run("systemctl start postgresql.service")

class DBHandler:
    def __enter__(self):
        stop_database()
        return self
    
    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()
        
def db_backup():
    run("pg_dump database")
    
def main():
    with DBHandler():
        db_backup()

이 예제에서는 DBHandler를 사용한 블록 내부에서 컨텍스트 관리자의 결과를 사용하지 않았다. 적어도 이 경우에 '\__enter\__'의 반환 값은 쓸모가 없다. 컨텍스트 관리자를 디자인할 때 블록이 시작된 후에 무엇이 필요한지 고려해야 한다. 일반적으로 필수는 아니지만 '\__enter\__'에서 무언가를 반환하는 것이 좋은 습관이다.

main() 함수에서는 유지보수 작업과 상관 없이 백업을 실행한다. 또한 백업에 오류가 있어도 여전히 '\__exit\__'을 호출한다.

'\__exit\__' 메서도의 서명을 주목할 필요가 있다. 블록에서 발생한 예외를 파라미터로 받는다. 블록에 예외가 없으면 모두 None이다.

'\__exit\__'의 반환 값을 잘 생각해야 한다. 특별한 작업을 할 필요가 없다면 아무것도 반환하지 않아도 된다. 만약 '\__exit\__'가 True를 반환하면 잠재적으로 발생한 예외를 호출자에게 전파하지 않고 멈춘다는 것을 뜻한다. 때로는 이렇게 하는 것을 원하는 경우도 있지만 일반적으로 발생된 예외를 삼키는 것은 좋지 않은 습관이다. 절대 오류를 조용히 무시해버리면 안 된다는 점을 기억하자.

### 컨텍스트 관리자 구현

앞의 예제와 같은 방법으로 컨텍스트 관리자를 구현할 수 있다. '\__enter\__'와 '\__exit\__' 매직 메서드만 구현하면 해당 객체는 컨텍스트 관리자 프로토콜을 지원할 수 있다. 이렇게 컨텍스트 관리자를 구현하는 것이 일반적인 방법이지만 유일한 방법은 아니다.

컨텍스트 관리자를 좀 더 간결하게 구현하는 방법뿐만 아니라 표준 라이브러리 특히 contextlib 모듈을 사용하여 보다 쉽게 구현하는 방법을 살펴볼 것이다. 

contextlib 모듈은 컨텍스트 관리자를 구현하거나 더 간결한 코드를 작성하는 데 도움이 되는 많은 모우미 함수와 객체를 제공한다. 

먼저 contextmanager 데코레이터를 살펴보자.

함수에 contextlib.contextmanager 데코레이터를 적용하면 해당 함수의 코드를 컨텍스트 관리자로 변환한다. 함수는 제너레이터라는 특수한 함수의 형태여야 하는데 이 함수는 코드의 문장을 '\__enter\__'와 '\__exit\__' 매직 메서드로 분리한다. 

지금은 데코레이터와 제너레이터에 익숙하지 않아도 다음에 살펴볼 예제는 특별한 설명 없이도 이해할 수 있고 관용구를 적용하여 이해할 수 있다. 이전 예제와 동일한 코드르 contextmanager 데코레이터를 사용해 다음과 같이 다시 작성할 수 있다.

In [18]:
import contextlib
def run(text):
    print(text)

@contextlib.contextmanager
def db_handler():
    stop_database()
    yield
    start_database()

with db_handler():
    db_backup()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


먼저 제너레이터 함수를 정의하고 @contextlib.contextmanager 데코레이터를 적용했다. 이 함수는 yield 문을 사용했으므로 제너레이터 함수가 된다. 여기서도 제너레이터의 상세 내용은 몰라도 된다. 중요한 것은 데코레이터를 적용하면 yield문 앞의 모든 것은 '\__enter\__' 메서드의 일부처럼 취급된다는 것이다. 여기서 생성된 값은 컨텍스트 관리자의 평가 결과로 사용된다. 이번 경우는 yield문에서 아무것도 반환하지 않았다. 이것은 암묵적으로 None을 반환하는 것과 같다. 

이 지점에서 제너레이터 함수가 중단되고 컨텍스트 관리자로 진입하여 데이터베이스의 백업 코드가 실행된다. 이 작업이 완료되면 다음 작업이 이어서 실행되므로 yield문 다음에 오는 모든 것들을 '\__exit\__' 로직으로 볼 수 있다.

이렇게 컨텍스트 매니저를 작성하면 기존 함수를 리팩토링하기 쉬운 장점이 있다. 일반적으로 어느 특정 객체에도 속하는 않는 컨텍스트 관리자가 필요한 경우 좋은 방법이다. 매직 메서드를 추가하면 업무 도메인에 보다 얽히게 되며, 책임이 커지고, 어쩌면 하지 않아도 될 것들을 지원해야만 한다. 많은 상태를 관리ㅏㄹ 필요가 없고 다른 클래스와 독립되어 있는 컨텍스트 관리자 함수를 만드는 경우는 이렇게 하는 것이 좋은 방법이다.

또 다른 도우미 클래스는 contextlib.ContextDecorator이다. 이 클래스는 컨텍스트 관리자 안에서 실행될 함수에 데코레이터를 적용하기 위한 로직을 제공하는 믹스인 클래스이다. 반면에 컨텍스트 관리자 자체의 로직은 앞서 언급한 매직 메서드를 구현하여 제공해야 한다. 

In [20]:
class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()
    
    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()

@dbhandler_decorator()
def offline_backup():
    run("pg_dump database")

이전 예제와 다른 점은 with문이 없다는 것이다. 그저 함수를 호출하기만 하면 offline_backup 함수가 컨텍스트 관리자 안에서 자동으로 실행된다. 이것이 원본 함수를 래핑하는 데코레이터가 하는 일이다.

이 접근법의 유일한 단점은 완전히 독립적인 것이라는 것이다. 이것은 좋은 특성이다. 데코레이터는 함수에 대해 아무것도 모르고 그 반대도 마찬가지이다. 이것은 좋은 특징이지만 컨텍스트 관리자 내부에서 사용하고자 하는 객체를 얻을 수 없다는 것을 의미한다.

데코레이터의 이점은 로직을 한번만 정의하면 동일한 로직을 필요로 하는 함수에 단지 데코레이터를 적용함으로써 원하는 만큼 재사용할 수 있다.

### 프로퍼티, 속성과 객체 메서드의 다른 타입들

public, private, protected 프로퍼티를 가지는 다른 언어들과는 다르게 파이썬 객체의 모든 프로퍼티와 함수는 public이다. 즉 호출자가 객체의 속성을 호출하지 못하도록 할 방법이 없다.

엄격한 강제사항은 없지만 몇 가지 규칙이 있다. 밑줄로 시작하는 속성은 해당 객체에 대해 pricate을 의미하여, 외부에서 호출하지 않기를 기대하는 것이다. 그러나 금지하는 것은 아니다.

파이썬 밑줄의 관습을 알아보자.

### 파이썬에서의 밑줄

In [22]:
class Connector:
    def __init__(self, source):
        self.source = source
        self._timeout = 60

conn = Connector("postgresql://localhost")

'postgresql://localhost'

In [23]:
conn.source

'postgresql://localhost'

In [24]:
conn._timeout

60

In [25]:
conn.__dict__

{'source': 'postgresql://localhost', '_timeout': 60}

_timeout은 connector 차제에서만 사용되고 호출자는 이 속성에 접근하지 않아야 한다. 

timeout 속성은 내부에서만 사용되고 바깥에서는 호출되지 않아서 동일한 인터페이스를 유지하므로 언제든 필요한 경우에 안전하게 리팩토링할 수 있어야 한다. 

이러한 규칙을 준수하면 객체의 인터페이스를 유지하였으므로 파급 효과에 대해 걱정하지 않아도 되기 때문에 유지보수가 쉽고 보다 견고한 코드를 작성할 수 있다. 동이한 원칙이 메서드에도 적용된다.

아래는 객체의 인터페이스를 명확하게 구분하기 위한 파이썬스러운 방식이다. 그러나 일부 속성과 메서드를 실제로 private으로 만들 수 있다는 오해가 있다. 다시 말하지만 이것은 오해이다. 이제 timeoout 속성을 이중 밑줄로 정의했다고 가정해보자. 

In [26]:
class Connector:
    def __init__(self, source):
        self.source = source
        self.__timeout = 60
    
    def connect(self):
        print("connecting with {0}s" .format(self.__timeout))
conn = Connector("postgresql://localhost")
conn.connect()

connecting with 60s


In [27]:
conn.__timeout

AttributeError: 'Connector' object has no attribute '__timeout'

일부 개발자는 이 예제와 같은 방법을 사용하여 일부 속성을 숨길 수 있으므로 timeout이 이제 private이며 다른 객체가 수정할 수 없다고 생각하느 경우가 있다. 그러나 위의 에러를 보면 속성이 존재하지 않는다는 의미다. 프라이빗이다 혹은 접근할 수 없다가 아니라 존재하지 않는다고 말한다. 

밑줄 두 개를 사용하면 실제로 파이썬은 다른 이름을 만든다. 이름 맹글링(mangling)이라 한다. "_<class-name\>__<attribute-name\>" 경우

"_Connector__timeout"이라는 속성이 만들어지며 이러한 속성은 다음과 같이 접근하여 수정할 수 있다.

In [28]:
vars(conn)

{'source': 'postgresql://localhost', '_Connector__timeout': 60}

In [29]:
conn._Connector__timeout

60

In [30]:
conn._Connector__timeout = 30

In [31]:
conn.connect()

connecting with 30s


앞에서 언급한 부작용에 의해 속성이 다른 이름으로 존재하는 것에 주목하자. 때문에 앞의 예제에서 속성에러가 발생하는 것이다. 

파이썬에서 이중 밑줄을 사용하는 것은 완전히 다른 경우이다. 여러번 확장되는 클래스의 메서드를 이름 충돌 없이 오버라이드하기 위해 만들어졌다. 

이중 밑줄은 파이썬스러운 코드가 아니다. 속성을 private으로 정의하려는 경우 하나의 밑줄을 사용하고 파이썬스러운 관습을 지키도록 해야 한다.

### 프로퍼티

객체에 값을 저장해야 할 경우 일반적인 속성을 사용할 수 있다. 때로는 객체의 상태나 다른 속성의 값을 기반으로 어떤 계산을 하려고 할 때도 있다. 이런 경우 대부분 프로퍼티를 사용하는 것이 좋은 선택이다.

프로퍼티는 객체의 어떤 속성에 대한 접근을 제어하려는 경우 사용한다. 이러게 하는 것 또한 파이썬스러운 코드이다. 자바와 같은 다른 프로그래밍 언어에서는 접근 메서드(getter, setter)를 만들지만 파이썬에서는 프로퍼티를 사용한다. 

사용자가 등록한 정보에 잘못된 정보가 입력되지 않게 보호하려고 한다고 해보자. 예를 들어 다음과 같이 이메일을 입력하는 경우이다.

In [35]:
import re

EMAIL_FORMAT = re.compile(r"[^@]+@[^@]+[^@]+")

def is_valid_email(potentially_valid_email: str):
    return re.match(EMAIL_FORMAT, potentially_valid_email) is not None

class User:
    def __init__(self, username):
        self.username = username
        self._email = None
        
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, new_email):
        if not is_valid_email(new_email):
            raise ValueError(f"유효한 이메일이 아니므로 {new_email} 값을 사용할 수 없음")
        self._email = new_email

이메일에 프로퍼티를 사용하여 공짜로 몇 가지 이점을 얻을 수 있다. 이 예제에서 첫 번째 @property 메서드는 private 속성인 email 값을 반환한다. 앞에서 언급했듯이 맨 앞에 있는 밑줄은 이 속성이 private으로 사용될 것이므로 외부에서 접근하면 안 된다는 뜻이다.

두 번째 메서트는 앞에서 정의한 프로퍼티에 @email.setter를 추가한다. 이 메서드는 user.email = new_email이 실행될 때 호출되는 코드로 new_email이 파라미터이다. 

In [38]:
u1 = User('llel')
u1.email = "llel@"

ValueError: 유효한 이메일이 아니므로 llel@ 값을 사용할 수 없음

In [39]:
u1.email = 'llel@llel'

In [40]:
u1.email

'llel@llel'

이렇게 하면 get_, set_ 접두어를 사용하여 사용자 메서드를 만드는 것보다 훨씬 더 간단하다. 

단 객체의 모든 속성에 대해 get, set 메서드를 작성할 필요가 없다. 대부분의 경우 일반 속성을 사용하는 것으로 충분하다. 소성 값을 가져오거나 수정할 때 특별한 로직이 필요한 경우에만 프로퍼티를 사용하자.

프로퍼티는 명령-쿼리 분리 원칙을 따르기 위한 좋은 방법이다. 명령-쿼리 분리 원칙은 '객체의 메서드'가 무언가의 상태를 변경하는 커맨드이거나 무언가의 값을 반환하는 쿼리이거나 둘 중 하나만 수행해야지 둘 다 동시에 수행하면 안 된다는 것이다.

@property 데코레이터는 무언가에 응답하기 위한 쿼리이고, @property.setter는 무언가를 하기 위한 커맨드이다. 

한 메서드에서 한 가지 이상의 일을 하지 말라는 것도 알아두자. 유효성 검사를 하고 싶으면 두 개 이상의 문장으로 나누어야 한다. 
한 가지만 수행해야 작업을 처리한 다음 상태를 확인하려면 메서드를 분리해야 한다.

### 이터러블 객체

파이썬에는 기본적으로 반복 가능한 객체가 있다. 리스트 튜플, 세트, 사전은 원하는 구조의 테이터를 보유할 수 있을 뿐 아니라 for문으로 반복적으로 가져올 수 있다. 

그러나 이러한 내장 반복형 객체만 for루프에서 사용 가능한 것은 아니다. 반복을 위해 정의한 로직을 사용해 자체 이터러블을 만들 수도 있다. 엄밀히 말하면 이터러블은 '\__iter__' 매직 메서드를 구현한 객체, 이터레이터는 '\__next__' 매직 메서드를 구현한 객체를 말하는데 지금은 반복과 관련된 객체 정도로만 이해한다.(매직 메서드란? :클래스안에 정의할 수 있는 스페셜 메소드 빌트인 타입(int, str)과 같은 동작을 하게 해준다. 연산자에 대해 데이터 타입에 맞는 메서드로 오버로딩한다. 메서드 이름 앞 뒤로 더블언더스코어를 붙인다.

파이썬의 반복은 이터러블 프로토콜이라는 자체 프로토콜을 사용해 동작한다.

for e in myobj: 형태로 객체를 반복할 수 있는지 확인하기 위해 파이썬은 고수준에서 다음 두 가지를 차례로 검사한다.
    1. 객체가 '__next__'나 '__iter__' 이터레이터 메서드 중 하나를 포함하는지 여부 
    2. 객체가 시퀀스이고 '__len__'과 '__getitem__'를 모두 가졌는지 여부
    
### 이터러블 객체 만들기